In [2]:
import pandas as pd
import numpy as np
import zipfile
import re
from datetime import datetime

In [3]:
import tweepy
TWITTER_KEY = '5DyjFhUosxuU24UKVK8huSS3L'
TWITTER_SECRET = 'VCrwDrGOUPwX0QyhbS4GLxlgJ9Jf3gBetU1l71c6Y0PulDYeX2'

In [3]:
# import pandas as pd
# import zipfile

# zf = zipfile.ZipFile('/home/sunny/fall_2020/ee460j/data/uselection_tweets_1jul_11nov.zip') 
# df = pd.read_csv(zf.open('uselection_tweets_1jul_11nov.csv'))

In [4]:
# # columns we wish to keep/filter on
# cols_to_keep = [‘id’, ‘member_id’, ‘loan_amnt’, ‘funded_amnt’]
# # setup dataframe iterator, the ‘usecols’ parameter filters the columns in the csv
# df_iter = pd.read_csv(r’/tmp/z_data/LoanStats_2016Q1.csv.zip’, skiprows=1, compression=’zip’,
# chunksize=20000, usecols=cols_to_keep)
# dfs = [] # this list will store the filtered dataframes for concatenation
# for df in df_iter:
# temp_df = (df.rename(columns={col: col.lower()
# for col in df.columns})
# # filter
# .pipe(lambda x: x[x.funded_amnt > 10000]))
# dfs += [temp_df.copy()]
# # combine filtered dfs into large output df
# concat_df = pd.concat(dfs) 

SyntaxError: invalid character in identifier (<ipython-input-4-0dda888cd49f>, line 2)

In [5]:
leaning_republican = [
    "PatriotByGod",
    "donaldjrtrain",
    "baddiesfortrump",
    "Metalivy74",
    "RealMattCouch",
    "TomiLahren",
    "AZGOP",
    "seanhannity",
    "LouDobbs",
    "Jim_Jordan",
    "tedcruz",
    "DineshDSouza",
    "GOP",
    "kelliwardaz",
    "PressSec",
    "_MeganPowers",
    "TrumpWarRoom",
    "VPComDir",
    "JasonMillerinDC",
    "BorisEP",
    "RepublicanChick"
]

leaning_democrat = [
    "SenKamalaHarris",
    "PreetBharara",
    "texasdemocrats",
    "BetoORourke",
    "AOC",
    "RepAdamSchiff",
    "PortiaEconomics",
    "MzKatieCassidy",
    "DrBiden",
    "sabrinasingh24",
    "DouglasEmhoff",
    "TeamJoe",
    "MichaelLaRosaDC",
    "BidenOhio",
    "SenWarren",
    "ewarren",
    "MollyJongFast",
    "votevets",
    "XavierBecerra",
    "Women4Biden",
    "Il4Kamala",
    "BernieSanders",
    "WhiteHouseWacko"
]

awesome_guy = [
    'kharel_sunny'
]

In [6]:
auth = tweepy.AppAuthHandler(TWITTER_KEY, TWITTER_SECRET)
api = tweepy.API(auth) 

test_users = api.lookup_users(screen_names = ['BernieSanders'] )
test_users

[User(_api=<tweepy.api.API object at 0x7fdbacc3d790>, _json={'id': 216776631, 'id_str': '216776631', 'name': 'Bernie Sanders', 'screen_name': 'BernieSanders', 'location': 'Vermont', 'description': 'U.S. Senator for Vermont. Not me, us.', 'url': 'https://t.co/jpg8Sp1GhR', 'entities': {'url': {'urls': [{'url': 'https://t.co/jpg8Sp1GhR', 'expanded_url': 'https://BernieSanders.com', 'display_url': 'BernieSanders.com', 'indices': [0, 23]}]}, 'description': {'urls': []}}, 'protected': False, 'followers_count': 13951002, 'friends_count': 1480, 'listed_count': 22434, 'created_at': 'Wed Nov 17 17:53:52 +0000 2010', 'favourites_count': 966, 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'verified': True, 'statuses_count': 19248, 'lang': None, 'status': {'created_at': 'Wed Dec 02 19:12:26 +0000 2020', 'id': 1334213856151957505, 'id_str': '1334213856151957505', 'text': 'Mitch McConnell\'s "new" COVID relief bill gives CEOs a 100% tax deduction for a 3-martini lunch, but ZERO to the 26

In [12]:
test_users[0]._json

{'id': 216776631,
 'id_str': '216776631',
 'name': 'Bernie Sanders',
 'screen_name': 'BernieSanders',
 'location': 'Vermont',
 'description': 'U.S. Senator for Vermont. Not me, us.',
 'url': 'https://t.co/jpg8Sp1GhR',
 'entities': {'url': {'urls': [{'url': 'https://t.co/jpg8Sp1GhR',
     'expanded_url': 'https://BernieSanders.com',
     'display_url': 'BernieSanders.com',
     'indices': [0, 23]}]},
  'description': {'urls': []}},
 'protected': False,
 'followers_count': 13951002,
 'friends_count': 1480,
 'listed_count': 22434,
 'created_at': 'Wed Nov 17 17:53:52 +0000 2010',
 'favourites_count': 966,
 'utc_offset': None,
 'time_zone': None,
 'geo_enabled': True,
 'verified': True,
 'statuses_count': 19248,
 'lang': None,
 'status': {'created_at': 'Wed Dec 02 19:12:26 +0000 2020',
  'id': 1334213856151957505,
  'id_str': '1334213856151957505',
  'text': 'Mitch McConnell\'s "new" COVID relief bill gives CEOs a 100% tax deduction for a 3-martini lunch, but ZERO to the 26… https://t.co/Fp

In [24]:
import csv

def usernames_to_user_data(user_names, party):
    clean_users_keys = ['description', 'profile_image_url', 'id', 'name', 'screen_name',\
                    'location'] 
    users = api.lookup_users(screen_names=user_names) 
    clean_users = [ 
        { k: user._json[k] for k in clean_users_keys }
        for user in users
    ]
    for clean_user in clean_users:
        clean_user['party'] = party
    return clean_users

def users_to_csv(users, csv_file):
    csv_columns = ['description', 'profile_image_url', 'id', 'name', 'screen_name',\
                    'location', 'party'] 
    try:
        with open(csv_file, 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
            writer.writeheader()
            for user in users:
                writer.writerow(user)
    except IOError:
        print("I/O error")

In [38]:
republican_users = usernames_to_user_data(leaning_republican, 'republican')
republican_users

In [21]:
assert len(leaning_republican) == len(republican_users)

In [25]:
users_to_csv(republican_users, 'republican_users.csv')

In [39]:
df = pd.read_csv('republican_users.csv')
df.head()

,description,profile_image_url,id,name,screen_name,location,party
0,Melting #SnowFlakes with the fire of Patriotis...,http://pbs.twimg.com/profile_images/9903511655...,3389073989,GodGuns&Trump 🇺🇸,PatriotByGod,"Las Vegas, NV",republican
1,"100%Trump Supporter, I Follow Back #MAGA #Chri...",http://pbs.twimg.com/profile_images/1125263585...,1005373425819553792,#Veteran #AmericaFirst #MAGA #Trump2020,donaldjrtrain,"Indiana, USA",republican
2,intelligent women vote for @realdonaldtrump ••...,http://pbs.twimg.com/profile_images/1324136700...,1318619325402591232,TRUMP 2020,baddiesfortrump,NaN,republican
3,"Wife to my soul mate, Mom, Craft beer, Great ...",http://pbs.twimg.com/profile_images/1331323758...,985728510,Metal Ivy,Metalivy74,Wyoming/Colorado,republican
4,Christian - Father - Investigator - Political ...,http://pbs.twimg.com/profile_images/1330368465...,601535938,Matt Couch,RealMattCouch,"NW Arkansas, USA",republican


In [36]:
democratic_users = usernames_to_user_data(leaning_democrat, 'democrat')
assert len(leaning_democrat) == len(democratic_users)
users_to_csv(democratic_users, 'democratic_users.csv')

In [40]:
df = pd.read_csv('democratic_users.csv')
df.head()

,description,profile_image_url,id,name,screen_name,location,party
0,United States Senator for California,http://pbs.twimg.com/profile_images/9746909066...,803694179079458816,Kamala Harris,SenKamalaHarris,NaN,democrat
1,Patriotic American & proud immigrant. @Springs...,http://pbs.twimg.com/profile_images/1294804118...,834081571493785600,Preet Bharara,PreetBharara,"New York, NY",democrat
2,The official account of the Texas Democratic P...,http://pbs.twimg.com/profile_images/1013850694...,68219213,Texas Democrats,texasdemocrats,Texas,democrat
3,NaN,http://pbs.twimg.com/profile_images/1177725147...,342863309,Beto O'Rourke,BetoORourke,"El Paso, TX",democrat
4,"US Representative,NY-14 (BX & Queens). In a mo...",http://pbs.twimg.com/profile_images/9232748811...,138203134,Alexandria Ocasio-Cortez,AOC,"Bronx + Queens, NYC",democrat
